In [2]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data
warnings.filterwarnings('ignore')

In [3]:
ticker = "NG=F"
period = "5y"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [10]:
bt_sma = Backtest(data, SmaCross, cash=10_000, commission=0.004, trade_on_close=False)
stats = bt_sma.run()
stats

Start                     2019-04-09 00:00...
End                       2024-04-09 00:00...
Duration                   1827 days 00:00:00
Exposure Time [%]                   71.904762
Equity Final [$]                  29607.99125
Equity Peak [$]                  42618.095352
Return [%]                         196.079912
Buy & Hold Return [%]              -30.307517
Return (Ann.) [%]                   24.245821
Volatility (Ann.) [%]               79.850834
Sharpe Ratio                         0.303639
Sortino Ratio                        0.623526
Calmar Ratio                         0.497163
Max. Drawdown [%]                  -48.768321
Avg. Drawdown [%]                  -11.638335
Max. Drawdown Duration      377 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                     152.685594
Worst Trade [%]                     -31.20807
Avg. Trade [%]                    

In [11]:
bt_sma.plot()

Row(id='3058', ...)

In [9]:
bt_sma._strategy.__dict__

mappingproxy({'__module__': 'src.strategies',
              '__doc__': 'Simple moving average with crossover strategy.',
              'n1': 42,
              'n2': 252,
              'init': <function src.strategies.SmaCross.init(self)>,
              'next': <function src.strategies.SmaCross.next(self)>,
              '__abstractmethods__': frozenset(),
              '_abc_impl': <_abc._abc_data at 0x11d426580>})

In [6]:
%%time
stats = bt_sma.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

CPU times: user 3.04 s, sys: 13.9 ms, total: 3.06 s
Wall time: 3.06 s


Start                     2019-04-09 00:00...
End                       2024-04-09 00:00...
Duration                   1827 days 00:00:00
Exposure Time [%]                   91.666667
Equity Final [$]                 22118.568606
Equity Peak [$]                  45175.284794
Return [%]                         121.185686
Buy & Hold Return [%]              -30.307517
Return (Ann.) [%]                   17.206421
Volatility (Ann.) [%]               89.702146
Sharpe Ratio                         0.191817
Sortino Ratio                        0.400415
Calmar Ratio                         0.285891
Max. Drawdown [%]                  -60.185298
Avg. Drawdown [%]                  -14.721484
Max. Drawdown Duration      673 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                   32
Win Rate [%]                           46.875
Best Trade [%]                      75.786438
Worst Trade [%]                    -29.826944
Avg. Trade [%]                    

In [16]:
bt_sma.plot()

Row(id='2723', ...)

In [20]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=False)
stats = bt_momentum.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

CPU times: user 787 ms, sys: 4.96 ms, total: 792 ms
Wall time: 790 ms


Start                     2023-04-06 00:00...
End                       2024-04-06 00:00...
Duration                    366 days 00:00:00
Exposure Time [%]                   97.820163
Equity Final [$]                 18541.661499
Equity Peak [$]                  20936.762329
Return [%]                          85.416615
Buy & Hold Return [%]               81.159673
Return (Ann.) [%]                   84.793779
Volatility (Ann.) [%]                85.26015
Sharpe Ratio                          0.99453
Sortino Ratio                        3.187883
Calmar Ratio                          3.08072
Max. Drawdown [%]                  -27.524014
Avg. Drawdown [%]                   -5.684425
Max. Drawdown Duration      207 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                  359
Win Rate [%]                        50.417827
Best Trade [%]                      12.288746
Worst Trade [%]                    -11.259537
Avg. Trade [%]                    

In [21]:
bt_momentum.plot()

Row(id='3549', ...)

In [23]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=True)
stats = bt_momentum.run()
stats

Start                     2023-04-06 00:00...
End                       2024-04-06 00:00...
Duration                    366 days 00:00:00
Exposure Time [%]                   98.637602
Equity Final [$]                 11978.875977
Equity Peak [$]                  12417.980469
Return [%]                           19.78876
Buy & Hold Return [%]               81.159673
Return (Ann.) [%]                   19.670948
Volatility (Ann.) [%]               53.717595
Sharpe Ratio                         0.366192
Sortino Ratio                        0.677488
Calmar Ratio                         0.511346
Max. Drawdown [%]                  -38.468923
Avg. Drawdown [%]                  -11.403995
Max. Drawdown Duration      213 days 00:00:00
Avg. Drawdown Duration       52 days 00:00:00
# Trades                                  362
Win Rate [%]                        49.447514
Best Trade [%]                      10.243561
Worst Trade [%]                    -12.239237
Avg. Trade [%]                    